<a href="https://colab.research.google.com/github/V-Mulich/projects/blob/main/key_driver_price_chan_med_cent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ среднего чека в медицинском центре за 2021-2022 года

**Описание проекта:** \
Провести анализ изменения цены в медецинской клинике \
в 2022 году относительно 2021 года

**Цель проекта:** \
определить ключевые драйверы изменения цены лечения. \

**Задачи: **

